Version 9 : XEntropyLoss, 2 class

In [ ]:
import gc
import glob
import os
import json
import matplotlib.pyplot as plt
import pprint
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm
from PIL import Image
import re
import matplotlib
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import datasets, manifold, mixture, model_selection
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from xgboost import XGBClassifier
import cv2
from keras.applications.densenet import preprocess_input, DenseNet121
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D, MaxPooling1D, concatenate
import keras.backend as K
from sklearn.decomposition import TruncatedSVD, NMF

%matplotlib inline

In [ ]:
import os

print(os.listdir("../input/cute-cats-and-dogs-from-pixabaycom/"))
print(os.listdir("../input/cute-cats-and-dogs-from-pixabaycom/pixabay"))
print(os.listdir("../input/cute-cats-and-dogs-from-pixabaycom/pixabay/cats/"))
print(os.listdir("../input/cute-cats-and-dogs-from-pixabaycom/pixabay/dogs/"))

In [ ]:
!mkdir ../input/root ../input/root/pets ../input/root/pets/0 ../input/root/pets/1

In [ ]:
!cp ../input/cute-cats-and-dogs-from-pixabaycom/pixabay/cats/0/*.jpg ../input/root/pets/0/
!cp ../input/cute-cats-and-dogs-from-pixabaycom/pixabay/dogs/0/*.jpg ../input/root/pets/0/
!cp ../input/cute-cats-and-dogs-from-pixabaycom/pixabay/cats/1/*.jpg ../input/root/pets/1/
!cp ../input/cute-cats-and-dogs-from-pixabaycom/pixabay/dogs/1/*.jpg ../input/root/pets/1/

!cp ../input/cute-cats-and-dogs-from-pixabaycom/pixabay/cats/0/*.jpg ../input/root/pets/0/
!cp ../input/cute-cats-and-dogs-from-pixabaycom/pixabay/dogs/0/*.jpg ../input/root/pets/0/

!cp ../input/cute-cats-and-dogs-from-pixabaycom/pixabay/cats/1/*.jpg ../input/root/pets/1/
!cp ../input/cute-cats-and-dogs-from-pixabaycom/pixabay/dogs/1/*.jpg ../input/root/pets/1/

In [ ]:
labels = pd.read_csv("../input/cute-cats-and-dogs-from-pixabaycom/labels.csv")
print(labels.shape)
print(labels.id.nunique())
labels.head()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import functional as F
import random
import tensorflow as tf

import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt

def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.set_random_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

kaeru_seed = 1337
seed_everything(seed=kaeru_seed)
# ======================================

In [ ]:
data_dir = os.path.join('../input', 'root')
image_dataset = datasets.ImageFolder(os.path.join(data_dir, 'pets'))
# image_dataset = datasets.ImageFolder(os.path.join(data_dir))
print(len(image_dataset))  # 41272枚の訓練データ
image, label = image_dataset[0]  # 0番目の画像とラベル

In [ ]:
plt.figure()
plt.imshow(image)

t = transforms.RandomResizedCrop(224)
trans_image = t(image)

plt.figure()
plt.imshow(trans_image)

In [ ]:
batchsize = 32 
epochs = 20 
epoch_start =1

In [ ]:
data_dir = os.path.join('../input', 'root/')
# data_dir = os.path.join('../input', 'cute-cats-and-dogs-from-pixabaycom/pixabay/')
data_transform = transforms.Compose([
 transforms.Resize([224, 224]), transforms.RandomHorizontalFlip(),
 transforms.ToTensor(),
 transforms.Normalize((0.4914, 0.4822, 0.4465),
 (0.2023, 0.1994, 0.2010)), 
]) 
# full_Dataset = datasets.ImageFolder(data_dir,transform=data_transform) 
full_Dataset = datasets.ImageFolder(os.path.join(data_dir, 'pets'),transform=data_transform) 
# full_Dataset = datasets.ImageFolder(os.path.join(data_dir, 'cats'),transform=data_transform) 
print(len(full_Dataset)) 
train_size = int(0.8 * len(full_Dataset)) 
test_size = len(full_Dataset) - train_size 
train_Dataset, test_Dataset = torch.utils.data.random_split(full_Dataset, [train_size, test_size]) 
print(len(train_Dataset)) 
print(len(test_Dataset)) 
train_loader = torch.utils.data.DataLoader(train_Dataset,batch_size=batchsize, shuffle=True) 
test_loader = torch.utils.data.DataLoader(test_Dataset,batch_size=batchsize, shuffle=False) 
print("train images: {}".format(len(train_Dataset)))
print("test images: {}".format(len(test_Dataset))) 
print("epoch: {}".format(epochs))
print("batch size: {}".format(batchsize))

In [ ]:
print(full_Dataset.classes)
print(full_Dataset.class_to_idx)

GPU

In [ ]:
import torch.backends.cudnn as cudnn 

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("cuda is available!")
    cudnn.benchmark = True
    cudnn.deterministic = True

network

In [ ]:
print(os.listdir("../input/pytorch-pretrained-image-models/"))

In [ ]:
n_classes = 2
# n_classes = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

model = models.resnet34().to(device) 
model.load_state_dict(torch.load('../input/pytorch-pretrained-image-models/resnet34.pth'))

for p in model.parameters():
    p.requires_grad=False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, n_classes)
if use_gpu:
    model.cuda()

training functions

In [ ]:
def eval_net(model, data_loader, device="cpu"):
    model.eval()
    ys=[]
    ypreds=[]
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)
        
        with torch.no_grad():
            _, y_pred = model(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

In [ ]:
def train_net(model, train_loader, test_loader, only_fc=True, optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(), n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    if only_fc:
        optimizer = optimizer_cls(model.fc.parameters())
    else:
        optimizer = optimizer_cls(model.parameters())
    
    for epoch in range(n_iter):
        running_loss = 0.0
    
        model.train()
        n = 0
        n_acc = 0
        for i, (xx, yy) in tqdm(enumerate(train_loader), total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = model(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        train_acc.append(n_acc / n)
        val_acc.append(eval_net(model, test_loader, device))
        
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)

In [ ]:
model.to("cuda:0")

train_net(model, train_loader, test_loader, n_iter=20, device="cuda:0")

In [ ]:
# 保存
torch.save(model, 'cute_model')
# torch.save(model, 'binary_cute_model')

visualize

In [ ]:
def imshow(images, title=None):
    images = images.numpy().transpose((1, 2, 0))  # (h, w, c)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    images = std * images + mean
    images = np.clip(images, 0, 1)
    plt.imshow(images)
    if title is not None:
        plt.title(title)

images, classes = next(iter(train_loader))
print(images.size(), classes.size())  # torch.Size([4, 3, 224, 224]) torch.Size([4])
images = torchvision.utils.make_grid(images)
imshow(images)

make feature part

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import functional as F
import random
import tensorflow as tf

import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_profile_path(category):

    data = []

    for path in sorted(glob.glob('../input/petfinder-adoption-prediction/%s_images/*-1.jpg' % category)):

        data.append({
            'PetID': path.split('/')[-1].split('-')[0],
            'path': path,
        })
            
    return pd.DataFrame(data)

def resize_to_square(image, size):
    h, w, d = image.shape
    ratio = size / max(h, w)
    resized_image = cv2.resize(image, (int(w*ratio), int(h*ratio)), cv2.INTER_AREA)
    return resized_image

def image_to_tensor(image, normalize=None):
    tensor = torch.from_numpy(np.moveaxis(image / (255. if image.dtype == np.uint8 else 1), -1, 0).astype(np.float32))
    if normalize is not None:
        return F.normalize(tensor, **normalize)
    return tensor

def pad(image, min_height, min_width):
    h,w,d = image.shape

    if h < min_height:
        h_pad_top = int((min_height - h) / 2.0)
        h_pad_bottom = min_height - h - h_pad_top
    else:
        h_pad_top = 0
        h_pad_bottom = 0

    if w < min_width:
        w_pad_left = int((min_width - w) / 2.0)
        w_pad_right = min_width - w - w_pad_left
    else:
        w_pad_left = 0
        w_pad_right = 0

    return cv2.copyMakeBorder(image, h_pad_top, h_pad_bottom, w_pad_left, w_pad_right, cv2.BORDER_CONSTANT, value=(0,0,0))

class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, df, size):
        self.df = df
        self.size = size
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):

        row = self.df.iloc[idx]

        image = cv2.imread(row.path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = resize_to_square(image, self.size)
        image = pad(image, self.size, self.size)
        tensor = image_to_tensor(image, normalize={'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]})
            
        return tensor

In [ ]:
size = 224
train_image_files = get_profile_path('train')
test_image_files = get_profile_path('test')

In [ ]:
import itertools
import torch

# model = torch.load('../input/cute-model-cat/cute_model')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

def get_cute_feats(df, model):
    model.eval()
    
    cute_or_not = []

    dataset = Dataset(df, size)
    loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)
    
    for xx in tqdm(loader, total=len(loader)):
        xx = xx.to(device)
        h = model(xx).max(1)
        cute_or_not.append(h)
        
    li = []
    for i in range(len(cute_or_not)):
        li.append(list(cute_or_not[i][1].cpu().numpy()))

    features = [ flatten for inner in li for flatten in inner ]
    
    features = np.array(features)
    features = pd.DataFrame(features)
    features = features.add_prefix('cute_or_not_')
    features.loc[:,'PetID'] = df['PetID']
    
    return features

In [ ]:
train_cute = get_cute_feats(train_image_files, model)
print(train_cute.shape)
train_cute.head()

In [ ]:
test_cute = get_cute_feats(test_image_files, model)
print(test_cute.shape)
test_cute.head()

In [ ]:
# train_cute.to_csv('train_cute.csv', index=False)
# test_cute.to_csv('test_cute.csv', index=False)